<a href="https://colab.research.google.com/github/rexxar280903/Qml/blob/main/PennyLane/svm_classifcation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import torch
from torch.nn.functional import relu
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
import matplotlib.pyplot as plt
import pandas as pd
np.random.seed(42)
from sklearn.decomposition import PCA
from tqdm import tqdm



In [9]:

# # Load the dataset (adjust file paths accordingly)
# train_data = pd.read_csv(r"C:\Users\user\Videos\belajar\qiskit2\PennyLane\dataset\optical+recognition+of+handwritten+digits\optdigits.tes", header=None)
# test_data = pd.read_csv(r"C:\Users\user\Videos\belajar\qiskit2\PennyLane\dataset\optical+recognition+of+handwritten+digits\optdigits.tra", header=None)

import pandas as pd

# URL raw dari GitHub (ganti USERNAME dan REPO jika berbeda)
url_train = "https://raw.githubusercontent.com/rexxar280903/Qml/refs/heads/main/PennyLane/dataset/optical%2Brecognition%2Bof%2Bhandwritten%2Bdigits/optdigits.tra"
url_test = "https://raw.githubusercontent.com/rexxar280903/Qml/refs/heads/main/PennyLane/dataset/optical%2Brecognition%2Bof%2Bhandwritten%2Bdigits/optdigits.tes"

# Membaca dataset langsung dari URL
train_data = pd.read_csv(url_train, header=None)
test_data = pd.read_csv(url_test, header=None)

# Cek data berhasil dibaca
print(train_data.head())
print(test_data.head())



   0   1   2   3   4   5   6   7   8   9   ...  55  56  57  58  59  60  61  \
0   0   1   6  15  12   1   0   0   0   7  ...   0   0   0   6  14   7   1   
1   0   0  10  16   6   0   0   0   0   7  ...   0   0   0  10  16  15   3   
2   0   0   8  15  16  13   0   0   0   1  ...   0   0   0   9  14   0   0   
3   0   0   0   3  11  16   0   0   0   0  ...   0   0   0   0   1  15   2   
4   0   0   5  14   4   0   0   0   0   0  ...   0   0   0   4  12  14   7   

   62  63  64  
0   0   0   0  
1   0   0   0  
2   0   0   7  
3   0   0   4  
4   0   0   6  

[5 rows x 65 columns]
   0   1   2   3   4   5   6   7   8   9   ...  55  56  57  58  59  60  61  \
0   0   0   5  13   9   1   0   0   0   0  ...   0   0   0   6  13  10   0   
1   0   0   0  12  13   5   0   0   0   0  ...   0   0   0   0  11  16  10   
2   0   0   0   4  15  12   0   0   0   0  ...   0   0   0   0   3  11  16   
3   0   0   7  15  13   1   0   0   0   8  ...   0   0   0   7  13  13   9   
4   0   0   0   1  11 

In [10]:
train_data[0]

,0
0,0
1,0
2,0
3,0
4,0
...,...
3818,0
3819,0
3820,0
3821,0


In [11]:

# Extract features and labels
X_train = train_data.iloc[:, :-1].values  # Features
y_train = train_data.iloc[:, -1].values   # Labels
X_test = test_data.iloc[:, :-1].values    # Features
y_test = test_data.iloc[:, -1].values     # Labels

# Normalize data (important for quantum encoding)
X_train = X_train / np.max(X_train)
X_test = X_test / np.max(X_test)

# Reduce dimensionality using PCA
n_components = 4  # Reduce to 4 features (adjustable)
pca = PCA(n_components=n_components)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Define a quantum device
n_qubits = n_components  # Use reduced number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum embedding
@qml.qnode(dev)
def quantum_embedding(x):
    qml.AngleEmbedding(x, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights=np.ones((1, n_qubits)), wires=range(n_qubits))
    return qml.probs(wires=range(n_qubits))

# Compute the quantum kernel with progress tracking
def quantum_kernel(X1, X2):
    kernel_matrix = np.zeros((len(X1), len(X2)))
    for i in tqdm(range(len(X1)), desc="Computing Quantum Kernel"):
        for j in range(len(X2)):
            kernel_matrix[i, j] = np.dot(quantum_embedding(X1[i]), quantum_embedding(X2[j]))
    return kernel_matrix

# Compute the kernel matrix
kernel_matrix = quantum_kernel(X_train, X_train)

# Train the classical SVM with the quantum kernel
clf = SVC(kernel="precomputed")
clf.fit(kernel_matrix, y_train)

# Prediction function with progress tracking
def predict(X_test):
    kernel_test = np.zeros((len(X_test), len(X_train)))
    for i in tqdm(range(len(X_test)), desc="Computing Test Kernel"):
        for j in range(len(X_train)):
            kernel_test[i, j] = np.dot(quantum_embedding(X_test[i]), quantum_embedding(X_train[j]))
    return clf.predict(kernel_test)

# Test the classifier
predictions = predict(X_test)
accuracy = np.mean(predictions == y_test)
print("Accuracy:", accuracy)


Computing Quantum Kernel:   0%|          | 12/3823 [04:28<23:43:26, 22.41s/it]


KeyboardInterrupt: 